<a href="https://colab.research.google.com/github/Sarah-wookey/RL_NetHack_2020/blob/main/DQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
# install prerequisites for nle
!sudo apt-get install -y build-essential autoconf libtool pkg-config \
    python3-dev python3-pip python3-numpy git libncurses5-dev \
    libzmq3-dev flex bison

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
pkg-config is already the newest version (0.29.1-0ubuntu2).
python3-numpy is already the newest version (1:1.13.3-2ubuntu1).
python3-numpy set to manually installed.
git is already the newest version (1:2.17.1-1ubuntu0.7).
libncurses5-dev is already the newest version (6.1-1ubuntu1.18.04).
python3-dev is already the newest version (3.6.7-1~18.04).
libzmq3-dev is already the newest version (4.2.5-1ubuntu0.2).
The following additional packages will be installed:
  automake autotools-dev file libbison-dev libfl-dev libfl2 libmagic-mgc
  libmagic1 libsigsegv2 m4 python-pip-whl python3-asn1crypto
  python3-cffi-backend python3-crypto python3-cryptography python3-idna
  python3-keyring python3-keyrings.alt python3-pkg-resources
  python3-secretstorage python3-setuptools python3-six python3-wheel
  python3-xdg
Suggested packages:
  autoco

In [ ]:
# download, build and install flatbuffers

!git clone https://github.com/google/flatbuffers.git
# all these commands have to be run in the same directory and !cd doesn't change
# the directory permanently in colab see: 
# https://stackoverflow.com/questions/48298146/changing-directory-in-google-colab-breaking-out-of-the-python-interpreter
!cd flatbuffers && cmake -G "Unix Makefiles" && make && sudo make install

Cloning into 'flatbuffers'...
remote: Enumerating objects: 19411, done.
remote: Total 19411 (delta 0), reused 0 (delta 0), pack-reused 19411
Receiving objects: 100% (19411/19411), 11.73 MiB | 26.12 MiB/s, done.
Resolving deltas: 100% (13497/13497), done.
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Looking for strtof_l
-- Looking for strtof_l - found
-- Looking for strtoull_l
-- Looking for strtoull_l - found
-- `tests/monster_test.fbs`: add ge

In [ ]:
# the next step requires a version of cmake > 3.14.0
!pip install cmake==3.15.3

     |████████████████████████████████| 16.5MB 208kB/s 
  Found existing installation: cmake 3.12.0
    Uninstalling cmake-3.12.0:
      Successfully uninstalled cmake-3.12.0


In [ ]:
# add -v for verbose if there are any errors
!pip install nle==0.5.2

     |████████████████████████████████| 6.7MB 13.4MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Using cached https://files.pythonhosted.org/packages/00/84/fc9dc13ee536ba5e6b8fd10ce368fea5b738fe394c3b296cde7c9b144a92/pybind11-2.6.1-py2.py3-none-any.whl
  Created wheel for nle: filename=nle-0.5.2-cp36-cp36m-linux_x86_64.whl size=2836551 sha256=0aee4f052896be66d057f0e6b18b939dc76e3c165f9aa1f927567731ca3980e8
  Stored in directory: /root/.cache/pip/wheels/3c/e1/2e/ece6eaf3e2382f04888155cb96fcb005f94c988d5f0c315d00
Successfully built nle


In [ ]:
!pip install "nle[agent]"

In [ ]:
!pip install pyvirtualdisplay

# Imports

In [ ]:

import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
# %matplotlib inline
import math
import glob
import io
import os
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import multiprocessing as mp
import torch.distributions as distributions
from torchvision import datasets, transforms

from nle import nethack

from pyvirtualdisplay import Display
# display = Display(visible=0, size=(1400, 900))
# display.start()
from collections import defaultdict

Colab = True

if Colab:
    # Imports specifically for google colab/drive
    from google.colab import files, drive
    drive.mount('/content/gdrive')
    dir = '/content/gdrive/My Drive/Colab Notebooks/Nethack/'
    os.makedirs(dir, exist_ok=True)
else:
    dir = './'

path = lambda fname: os.path.join(dir, fname)

from tqdm import tqdm

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

# Crop

In [ ]:

class Crop(nn.Module):
    """Helper class for NetHackNet below."""

    def __init__(self, height, width, height_target, width_target):
        super(Crop, self).__init__()
        self.width = width
        self.height = height
        self.width_target = width_target
        self.height_target = height_target
        width_grid = _step_to_range(2 / (self.width - 1), self.width_target)[
            None, :
        ].expand(self.height_target, -1)
        height_grid = _step_to_range(2 / (self.height - 1), height_target)[
            :, None
        ].expand(-1, self.width_target)

        # "clone" necessary, https://github.com/pytorch/pytorch/issues/34880
        self.register_buffer("width_grid", width_grid.clone())
        self.register_buffer("height_grid", height_grid.clone())

    def forward(self, inputs, coordinates):
        """Calculates centered crop around given x,y coordinates.
        Args:
           inputs [B x H x W]
           coordinates [B x 2] x,y coordinates
        Returns:
           [B x H' x W'] inputs cropped and centered around x,y coordinates.
        """
        assert inputs.shape[1] == self.height, f'{ inputs.shape[1],self.height}'
        assert inputs.shape[2] == self.width

        inputs = inputs[:, None, :, :].float()

        x = coordinates[:, 0]
        y = coordinates[:, 1]

        x_shift = 2 / (self.width - 1) * (x.float() - self.width // 2)
        y_shift = 2 / (self.height - 1) * (y.float() - self.height // 2)

        grid = torch.stack(
            [
                self.width_grid[None, :, :] + x_shift[:, None, None],
                self.height_grid[None, :, :] + y_shift[:, None, None],
            ],
            dim=3,
        )

        # TODO: only cast to int if original tensor was int
        return (
            torch.round(F.grid_sample(inputs, grid, align_corners=True))
            .squeeze(1)
            .long()
        )

# Replay Buffer


In [ ]:
import os
import matplotlib as mp
if os.environ.get('DISPLAY','') == '':
    print('no display found. Using non-interactive Agg backend')
    mp.use('Agg')
import matplotlib.pyplot as plt

import numpy as np
import gym
import nle
import random
from gym import spaces

from collections import deque
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

"""replay buffer"""
class ReplayBuffer:
    """
    Simple storage for transitions from an environment.
    """

    def __init__(self, size):
        """
        Initialise a buffer of a given size for storing transitions
        :param size: the maximum number of transitions that can be stored
        """
        self._storage = []
        self._maxsize = size
        self._next_idx = 0

    def __len__(self):
        return len(self._storage)

    def add(self, state, action, reward, next_state, done):
        """
        Add a transition to the buffer. Old transitions will be overwritten if the buffer is full.
        :param state: the agent's initial state
        :param action: the action taken by the agent
        :param reward: the reward the agent received
        :param next_state: the subsequent state
        :param done: whether the episode terminated
        """
        data = (state, action, reward, next_state, done)
        # print("state type",state.type)
        if self._next_idx >= len(self._storage):
            self._storage.append(data)
        else:
            self._storage[self._next_idx] = data
        self._next_idx = (self._next_idx + 1) % self._maxsize

    def _encode_sample(self, indices):
        states, actions, rewards, next_states, dones = {}, [], [], [], []
        glyphs, blstats, glyphs_ns, blstats_ns = [], [], [], []
        ## come back and fix this
        for i in indices:
            data = self._storage[i]
            state, action, reward, next_state, done = data
            # states.append(dict_to_device(state))
            glyphs.append(state['glyphs'].to(device))
            blstats.append(state['blstats'].to(device))
            glyphs_ns.append(next_state['glyphs'].to(device))
            blstats_ns.append(next_state['blstats'].to(device))            
            actions.append(action)
            rewards.append(reward)
            # next_states.append(dict_to_device(next_state))
            dones.append(done)
        
        dones = torch.stack(dones)
        states = {'glyphs': torch.cat(glyphs), 'blstats': torch.cat(blstats)}
        next_states = {'glyphs': torch.cat(glyphs_ns), 'blstats': torch.cat(blstats_ns)}

        return (
            (states),
            (actions),
            (rewards),
            (next_states),
            (dones),
        )

    def sample(self, batch_size):
        """
        Randomly sample a batch of transitions from the buffer.
        :param batch_size: the number of transitions to sample
        :return: a mini-batch of sampled transitions
        """
        indices = np.random.randint(0, len(self._storage) - 1, size=batch_size)
        return self._encode_sample(indices)

def to_state(state):
    glyphs_matrix = state['glyphs'] # T x B x H x W
    agent_stat = state['blstats']
    agent_row = state['blstats'][1]
    agent_col = state['blstats'][0]

    around_agent = np.zeros([9,9])
    row = agent_row - 4
    col = agent_col -4
    for i in range(9):
        for j in range(9):
            if row>0 and row<glyphs_matrix.shape[0] and col>0 and col<glyphs_matrix.shape[1]:
                around_agent[i][j] = glyphs_matrix[row][col]
            col+=1
        col = agent_col -4
        row+=1
        
    return glyphs_matrix.copy(),around_agent.copy(),agent_stat.copy()
    
    
class AbstractAgent:
    """
    AbstractAgent

    """
    def __init__(self):
        raise NotImplementedError()

    def act(self, observation):
        raise NotImplementedError()


class MyAgent(AbstractAgent):
    def __init__(self, observation_space, action_space):
        self.observation_space = observation_space
        self.action_space = action_space
        # TODO Initialise your agent's models
        self.model = DQN(observation_space,action_space).to(device)

        # for example, if your agent had a Pytorch model it must be load here
        self.model.load_state_dict(torch.load( 'models/dqn_nethack.pth', map_location=torch.device(device)))

    def act(self, observation):
        # Perform processing to observation

        # TODO: return selected action
        state = format_observations(observation)
        # glyphs_matrix,around_agent,agent_stat = to_state(observation)
        # glyphs_matrix = torch.from_numpy(glyphs_matrix).type(torch.FloatTensor).to(device).unsqueeze(0).unsqueeze(0)/5991.0
        # agent_stat = torch.from_numpy(agent_stat).type(torch.FloatTensor).to(device).unsqueeze(0)
        # around_agent = torch.from_numpy(around_agent).type(torch.FloatTensor).to(device).unsqueeze(0).unsqueeze(0)/5991.0
        # state = (glyphs_matrix,around_agent,agent_stat)

        with torch.no_grad():
            outputs, core_state = self.model(state, core_state, dones)
            action = torch.argmax(outputs)
        return action.item()

class RandomAgent(AbstractAgent):
    def __init__(self, observation_space, action_space):
        self.observation_space = observation_space
        self.action_space = action_space

    def act(self, observation):
        return self.action_space.sample()


def run_episode(env):
    # create instance of MyAgent
    # from MyAgent import MyAgent
    agent = MyAgent(env.observation_space,env.action_space.n)

    done = False
    episode_return = 0.0
    state = env.reset()

    save_dir = './animations_nethack_dqn/'
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    try:
        env = gym.wrappers.Monitor(
        env, save_dir, video_callable=lambda episode_id: True)
    except gym.error.Error as e:
        print(e)

    while not done:
        # pass state to agent and let agent decide action
        action = agent.act(state)
        env.render()
        new_state, reward, done, _ = env.step(action)
        episode_return += reward
        state = new_state
    return episode_return


"""model"""

# DQN Agent

In [ ]:

class DQNAgent:
    def __init__(self,observation_space: spaces.Box,action_space,replay_buffer,lr,batch_size,gamma):
        """
        Initialise the DQN algorithm using the Adam optimiser
        :param action_space: the action space of the environment
        :param observation_space: the state space of the environment
        :param replay_buffer: storage for experience replay
        :param lr: the learning rate for Adam
        :param batch_size: the batch size
        :param gamma: the discount factor
        """

        self.grad_norm_clipping = 40.0

        self.action_space = action_space
        self.observation_space = observation_space
        self.dqn = DQN(observation_space,action_space)
        self.target = DQN(observation_space,action_space)
        self.criterion = nn.L1Loss()
        self.optimizer = optim.RMSprop(self.dqn.parameters(), lr=lr,eps=0.000001)
        self.dqn.to(device)
        self.target.to(device)
        self.replay_buffer = replay_buffer
        self.batch_size = batch_size
        self.gamma = gamma
        self.update_target_network()
        self.core_state = self.dqn.initial_state()

    def optimise_td_loss(self):
        """
        Optimise the TD-error over a single minibatch of transitions
        :return: the loss
        """
        states, actions, rewards, next_states, dones = self.replay_buffer.sample(self.batch_size)

        # actions = torch.from_numpy(np.array(actions)).to(device)
        # rewards = torch.from_numpy(np.array(rewards)).to(device)
        # dones = torch.from_numpy(np.array(dones)).to(device)
        
        # matrixs = torch.from_numpy(matrixs).type(torch.FloatTensor).to(device).unsqueeze(1)/5991.0
        # agent_stats = torch.from_numpy(agent_stats).type(torch.FloatTensor).to(device)
        # around_agents = torch.from_numpy(around_agents).type(torch.FloatTensor).to(device).unsqueeze(1)/5991.0
        # states = (matrixs,around_agents,agent_stats)
        
        # matrix_ps = torch.from_numpy(matrix_ps).type(torch.FloatTensor).to(device).unsqueeze(1)/5991.0
        # agent_stat_ps = torch.from_numpy(agent_stat_ps).type(torch.FloatTensor).to(device)
        # around_agent_ps = torch.from_numpy(around_agent_ps).type(torch.FloatTensor).to(device).unsqueeze(1)/5991.0
        # next_states = (matrix_ps,around_agent_ps,agent_stat_ps)
        # states = list_dict_to_device(states)
        # next_states =  dict_to_device(next_states)
        
        actions = list_to_device(actions)
        rewards = list_to_device(rewards)
       
        dones = list_to_device(dones)
        prediction, self.core_state = self.dqn(states, self.core_state, dones)
        current_q_value = prediction['policy_logits'].squeeze().gather(1,actions.unsqueeze(1))


        with torch.no_grad():
            target_prediction, _ = self.target(next_states, self.core_state, dones)
            target_prediction = target_prediction
            max_q,_ = torch.max(target_prediction['policy_logits'].squeeze(),1)
            target_q_value = rewards.squeeze() + ((1-dones.squeeze().float())*self.gamma*max_q.squeeze())
            target_q_value = target_q_value.unsqueeze(1)


        loss = nn.functional.smooth_l1_loss(current_q_value, target_q_value)
        total_loss = loss.clone().detach().requires_grad_(True)

        self.optimizer.zero_grad()
        total_loss.backward()
        nn.utils.clip_grad_norm_(self.dqn.parameters(), self.grad_norm_clipping)
        self.optimizer.step()

        return total_loss.item()
        
    def update_target_network(self):
        """
        Update the target Q-network by copying the weights from the current Q-network
        """
        # TODO update target_network parameters with policy_network parameters
        self.target.load_state_dict(self.dqn.state_dict())
    
    def act(self, state, done):
        """
        Select an action greedily from the Q-network given the state
        :param state: the current state
        :return: the action to take
        """ 
        # state = format_observations(state)
        # self.DQN(dict_to_device(state), list_to_device(self.agent_state))
        with torch.no_grad():
            outputs, self.core_state = self.dqn(dict_to_device(state), list_to_device(self.core_state), done.to(device))
            action = torch.argmax(outputs['action'])
        return action.item()

# Other 

In [ ]:
def _step_to_range(delta, num_steps):
    """Range of `num_steps` integers with distance `delta` centered around zero."""
    return delta * torch.arange(-num_steps // 2, num_steps // 2)

In [ ]:
def list_dict_to_device(list_dict):
    return [ dict_to_device(dict_) for dict_ in list_dict ]

In [ ]:
def dict_to_device(dic):
    for key, value in dic.items():
        dic[key] = dic[key].to(device)
    return dic

In [ ]:
def list_to_device(l):
    l = torch.stack([ t.to(device) for t in l ])
    return l

In [ ]:
def format_observations(observation, keys=("glyphs", "blstats")):
    observations = {}
    for key in keys:
        entry = observation[key]
        entry = torch.from_numpy(entry)
        entry = entry.view((1, 1) + entry.shape)  # (...) -> (T,B,...).
        observations[key] = entry
    return observations

In [ ]:
def save_checkpoint(
    fname, agent, step_num,
    episode_rewards = None, num_episode_step_counter = None,
    train_loss=None, episode_loss=None, episode_loss_av = None,
    download=False):
    print('saving')
    checkpoint = {
        'observation_space': agent.observation_space,
        'step_num': step_num,
        'episode_rewards': episode_rewards,
        'train_loss': train_loss,
        'num_episode_step_counter':num_episode_step_counter,
        # 'episode_loss':episode_loss,
        # 'episode_loss_av':episode_loss_av,
        'replay_buffer': agent.replay_buffer,
        'model_state_dict': agent.dqn.state_dict(),
        'target_state_dict': agent.target.state_dict(),
        'optimizer_state_dict': agent.optimizer.state_dict(),
        'core_state': agent.core_state}

    torch.save(checkpoint, path(fname))
    if download and Colab: files.download(path) # not working well 

def load_checkpoint(fname, agent):
    # well, this function is just badly written
    # checkpoint = torch.load(path(fname), map_location=torch.device(device))
    checkpoint = torch.load(path(fname))
    agent.dqn.load_state_dict(checkpoint['model_state_dict'])
    #  self.model.load_state_dict(torch.load(PATH, map_location=torch.device(device)))
    agent.replay_buffer = (checkpoint['replay_buffer'])
    agent.target.load_state_dict(checkpoint['target_state_dict'])
    agent.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    agent.dqn.to(device)
    agent.target.to(device)
    agent.core_state = checkpoint['core_state']

    return checkpoint

# Network old

In [ ]:

# class DQN(nn.Module):
#     def __init__(
#         self,
#         observation_space,
#         num_actions,
#         use_lstm = True,
#         embedding_dim=32,
#         crop_dim=9,
#         num_layers=5
#     ):
#         """
#         Initialise the DQN
#         :param observation_space: the state space of the environment
#         :param action_space: the action space of the environment
#         """
#         super(DQN,self).__init__()

#         self.glyph_shape = observation_space["glyphs"].shape
#         self.blstats_size = observation_space["blstats"].shape[0]

#         self.num_actions = num_actions
#         self.use_lstm = use_lstm

#         self.H = self.glyph_shape[0]
#         self.W = self.glyph_shape[1]

#         self.k_dim = embedding_dim
#         self.h_dim = 512

#         self.crop_dim = crop_dim

#         self.crop = Crop(self.H, self.W, self.crop_dim, self.crop_dim)

#         self.embed = nn.Embedding(nethack.MAX_GLYPH, self.k_dim)

#         K = embedding_dim  # number of input filters
#         F = 3  # filter dimensions # kernel_size
#         S = 1  # stride
#         P = 1  # padding
#         M = 16  # number of intermediate filters
#         Y = 8  # number of output filters
#         L = num_layers  # number of convnet layers

#         self.extract_representation = nn.Sequential
#         (
#             nn.Conv2d(1, M, kernel_size=F, stride=S,padding = K),
#             nn.ELU(),
#             nn.Conv2d(M, M, kernel_size=F, stride=S,padding = K),
#             nn.ELU(),
#             nn.Conv2d(M, M, kernel_size=F, stride=S,padding = K),
#             nn.ELU(),
#             nn.Conv2d(M, M, kernel_size=F, stride=S,padding = K),
#             nn.ELU(),
#             nn.Conv2d(M, M, kernel_size=F, stride=S,padding = K),
#             nn.ELU(),
#         )

#         self.extract_crop_representation = nn.Sequential
#         (
#             nn.Conv2d(1, M, kernel_size=F, stride=S,padding = K),
#             nn.ELU(),
#             nn.Conv2d(M, M, kernel_size=F, stride=S,padding = K),
#             nn.ELU(),
#             nn.Conv2d(M, M, kernel_size=F, stride=S,padding = K),
#             nn.ELU(),
#             nn.Conv2d(M, M, kernel_size=F, stride=S,padding = K),
#             nn.ELU(),
#             nn.Conv2d(M, M, kernel_size=F, stride=S,padding = K),
#             nn.ELU(),
#         )

#         out_dim = self.k_dim # embedding_dimension = 32
#         out_dim += self.H * self.W * Y # CNN over full glyph map
#         out_dim += self.crop_dim ** 2 * Y # CNN crop model.

#         self.embed_blstats = nn.Sequential
#         (
#             nn.Linear(self.blstats_size, self.k_dim),
#             nn.ReLU(),
#             nn.Linear(self.k_dim, self.k_dim),
#             nn.ReLU(),
#         )

#         self.fc = nn.Sequential
#         (
#             nn.Linear(out_dim, self.h_dim),
#             nn.ReLU(),
#             nn.Linear(self.h_dim, self.h_dim),
#             nn.ReLU(),
#         )

#         if self.use_lstm:
#             self.core = nn.LSTM(self.h_dim, self.h_dim, num_layers=1)

#         self.policy = nn.Linear(self.h_dim, self.num_actions)
    

#     def initial_state(self, batch_size=1):
#         if not self.use_lstm:
#             return tuple()
#         return tuple((
#             torch.zeros(self.core.num_layers, batch_size, self.core.hidden_size),
#             torch.zeros(self.core.num_layers, batch_size, self.core.hidden_size)
#         ))

#     def forward(self, state, core_state, dones=None):
#         # -- [T x B x H x W] # 
#         glyphs = state["glyphs"]

#         # -- [T x B x F]
#         blstats = state["blstats"]

#         T, B, *_ = glyphs.shape

#         # -- [B' x H x W]
#         glyphs = torch.flatten(glyphs, 0, 1)  # Merge time and batch.

#         # -- [B' x F]
#         blstats = blstats.view(T * B, -1).float()

#         # -- [B x H x W]
#         glyphs = glyphs.long()
#         # -- [B x 2] x,y coordinates
#         coordinates = blstats[:, :2]
#         # TODO ???
#         # coordinates[:, 0].add_(-1)

#         # -- [B x F]
#         # FIXME: hack to use compatible blstats to before
#         # blstats = blstats[:, [0, 1, 21, 10, 11]]

#         blstats = blstats.view(T * B, -1).float()
#         # -- [B x K]
#         blstats_emb = self.embed_blstats(blstats)

#         assert blstats_emb.shape[0] == T * B

#         reps = [blstats_emb] # representations

#         # -- [B x H' x W']
#         crop = self.crop(glyphs, coordinates)

#         # print("crop", crop)
#         # print("at_xy", glyphs[:, coordinates[:, 1].long(), coordinates[:, 0].long()])

#         ## self.extract_crop_representation and self.extract_representation forward pass the same
#         # -- [B x H' x W' x K]
#         crop_emb = self._select(self.embed, crop)

#         # CNN crop model.
#         # -- [B x K x W' x H']
#         crop_emb = crop_emb.transpose(1, 3)  # -- TODO: slow?
#         # -- [B x W' x H' x K]
#         crop_rep = self.extract_crop_representation(crop_emb)

#         # -- [B x K']
#         crop_rep = crop_rep.view(T * B, -1)
#         assert crop_rep.shape[0] == T * B

#         reps.append(crop_rep)

#         # -- [B x H x W x K]
#         glyphs_emb = self._select(self.embed, glyphs)
#         # glyphs_emb = self.embed(glyphs)
#         # -- [B x K x W x H]
#         glyphs_emb = glyphs_emb.transpose(1, 3)  # -- TODO: slow?
#         # -- [B x W x H x K]
#         glyphs_rep = self.extract_representation(glyphs_emb)

#         # -- [B x K']
#         glyphs_rep = glyphs_rep.view(T * B, -1)
#         assert glyphs_rep.shape[0] == T * B

#         # -- [B x K'']
#         reps.append(glyphs_rep)

#         st = torch.cat(reps, dim=1)

#         # -- [B x K]
#         st = self.fc(st)

#         if self.use_lstm:
#             core_input = st.view(T, B, -1)
#             core_output_list = []
#             notdone = (~done).float()
#             for input, nd in zip(core_input.unbind(), notdone.unbind()):
#                 # Reset core state to zero whenever an episode ended.
#                 # Make `done` broadcastable with (num_layers, B, hidden_size)
#                 # states:
#                 nd = nd.view(1, -1, 1)
#                 core_state = tuple(nd * s for s in core_state)
#                 output, core_state = self.core(input.unsqueeze(0), core_state)
#                 core_output_list.append(output)
#             core_output = torch.flatten(torch.cat(core_output_list), 0, 1)
#         else:
#             core_output = st

#         # -- [B x A]
#         policy_logits = self.policy(core_output)
#         # -- [B x A]
        
#         print(F.softmax(policy_logits, dim=1))
#         if self.training:
#             action = torch.multinomial(F.softmax(policy_logits, dim=1), num_samples=1)
#         else:
#             # Don't sample when testing.
#             action = torch.argmax(policy_logits, dim=1)

#         policy_logits = policy_logits.view(T, B, self.num_actions)
        
#         action = action.view(T, B)

#         return (
#             dict(policy_logits=policy_logits,  action=action),
#             core_state,
#         )
# """agent"""

# Network

In [ ]:

class DQN(nn.Module):
    def __init__(
        self,
        observation_shape,
        num_actions,
        use_lstm = True,
        embedding_dim=32,
        crop_dim=9,
        num_layers=5
    ):
        """
        Initialise the DQN
        :param observation_space: the state space of the environment
        :param action_space: the action space of the environment
        """
        super(DQN,self).__init__()

        self.glyph_shape = observation_shape["glyphs"].shape
        self.blstats_size = observation_shape["blstats"].shape[0]

        self.num_actions = num_actions
        self.use_lstm = use_lstm

        self.H = self.glyph_shape[0]
        self.W = self.glyph_shape[1]

        self.k_dim = embedding_dim
        self.h_dim = 512

        self.crop_dim = crop_dim

        self.crop = Crop(self.H, self.W, self.crop_dim, self.crop_dim)

        self.embed = nn.Embedding(nethack.MAX_GLYPH, self.k_dim)

        K = embedding_dim  # number of input filters
        F = 3  # filter dimensions
        S = 1  # stride
        P = 1  # padding
        M = 16  # number of intermediate filters
        Y = 8  # number of output filters
        L = num_layers  # number of convnet layers

        in_channels = [K] + [M] * (L - 1)
        out_channels = [M] * (L - 1) + [Y]

        def interleave(xs, ys):
            return [val for pair in zip(xs, ys) for val in pair]

        conv_extract = [
            nn.Conv2d(
                in_channels=in_channels[i],
                out_channels=out_channels[i],
                kernel_size=(F, F),
                stride=S,
                padding=P,
            )
            for i in range(L)
        ]

        self.extract_representation = nn.Sequential(
            *interleave(conv_extract, [nn.ELU()] * len(conv_extract))
        )

        # CNN crop model.
        conv_extract_crop = [
            nn.Conv2d(
                in_channels=in_channels[i],
                out_channels=out_channels[i],
                kernel_size=(F, F),
                stride=S,
                padding=P,
            )
            for i in range(L)
        ]

        self.extract_crop_representation = nn.Sequential(
            *interleave(conv_extract_crop, [nn.ELU()] * len(conv_extract))
        )

        out_dim = self.k_dim
        # CNN over full glyph map
        out_dim += self.H * self.W * Y

        # CNN crop model.
        out_dim += self.crop_dim ** 2 * Y

        self.embed_blstats = nn.Sequential(
            nn.Linear(self.blstats_size, self.k_dim),
            nn.ReLU(),
            nn.Linear(self.k_dim, self.k_dim),
            nn.ReLU(),
        )

        self.fc = nn.Sequential(
            nn.Linear(out_dim, self.h_dim),
            nn.ReLU(),
            nn.Linear(self.h_dim, self.h_dim),
            nn.ReLU(),
        )

        if self.use_lstm:
            self.core = nn.LSTM(self.h_dim, self.h_dim, num_layers=1)

        self.policy = nn.Linear(self.h_dim, self.num_actions)
        self.baseline = nn.Linear(self.h_dim, 1)

    def _select(self, embed, x):
        # Work around slow backward pass of nn.Embedding, see
        # https://github.com/pytorch/pytorch/issues/24912
        out = embed.weight.index_select(0, x.reshape(-1))
        return out.reshape(x.shape + (-1,))

    def initial_state(self, batch_size=1):
        if not self.use_lstm:
            return tuple()
        return tuple((
            torch.zeros(self.core.num_layers, batch_size, self.core.hidden_size).to(device),
            torch.zeros(self.core.num_layers, batch_size, self.core.hidden_size).to(device)
        ))

    def forward(self, state, core_state, dones=None):
        # -- [T x B x H x W] # 
        glyphs = state["glyphs"]

        # -- [T x B x F]
        blstats = state["blstats"]

        T, B, *_ = glyphs.shape

        # -- [B' x H x W]
        glyphs = torch.flatten(glyphs, 0, 1)  # Merge time and batch.

        # -- [B' x F]
        blstats = blstats.view(T * B, -1).float()

        # -- [B x H x W]
        glyphs = glyphs.long()
        # -- [B x 2] x,y coordinates
        coordinates = blstats[:, :2]
        # TODO ???
        # coordinates[:, 0].add_(-1)

        # -- [B x F]
        # FIXME: hack to use compatible blstats to before
        # blstats = blstats[:, [0, 1, 21, 10, 11]]

        blstats = blstats.view(T * B, -1).float()
        # -- [B x K]
        blstats_emb = self.embed_blstats(blstats)

        assert blstats_emb.shape[0] == T * B

        reps = [blstats_emb] # representations

        # -- [B x H' x W']
        crop = self.crop(glyphs, coordinates)

        # print("crop", crop)
        # print("at_xy", glyphs[:, coordinates[:, 1].long(), coordinates[:, 0].long()])

        ## self.extract_crop_representation and self.extract_representation forward pass the same
        # -- [B x H' x W' x K]
        crop_emb = self._select(self.embed, crop)

        # CNN crop model.
        # -- [B x K x W' x H']
        crop_emb = crop_emb.transpose(1, 3)  # -- TODO: slow?
        # -- [B x W' x H' x K]
        crop_rep = self.extract_crop_representation(crop_emb)

        # -- [B x K']
        crop_rep = crop_rep.view(T * B, -1)
        assert crop_rep.shape[0] == T * B

        reps.append(crop_rep)

        # -- [B x H x W x K]
        glyphs_emb = self._select(self.embed, glyphs)
        # glyphs_emb = self.embed(glyphs)
        # -- [B x K x W x H]
        glyphs_emb = glyphs_emb.transpose(1, 3)  # -- TODO: slow?
        # -- [B x W x H x K]
        glyphs_rep = self.extract_representation(glyphs_emb)

        # -- [B x K']
        glyphs_rep = glyphs_rep.view(T * B, -1)
        assert glyphs_rep.shape[0] == T * B

        # -- [B x K'']
        reps.append(glyphs_rep)

        st = torch.cat(reps, dim=1)

        # -- [B x K]
        st = self.fc(st)

        if self.use_lstm:
            core_input = st.view(T, B, -1)
            core_output_list = []
            notdone = (dones)
            for input, nd in zip(core_input.unbind(), notdone.unbind()):
                # Reset core state to zero whenever an episode ended.
                # Make `done` broadcastable with (num_layers, B, hidden_size)
                # states:
                nd = nd.view(1, -1, 1)
                core_state = tuple(nd * s for s in core_state)
                output, core_state = self.core(input.unsqueeze(0), core_state)
                core_output_list.append(output)
            core_output = torch.flatten(torch.cat(core_output_list), 0, 1)
        else:
            core_output = st

        # -- [B x A]
        policy_logits = self.policy(core_output)
        # -- [B x A]
        
        # print(F.softmax(policy_logits, dim=1))
        if self.training:
            action = torch.multinomial(F.softmax(policy_logits, dim=1), num_samples=1)
        else:
            # Don't sample when testing.
            action = torch.argmax(policy_logits, dim=1)

        policy_logits = policy_logits.view(T, B, self.num_actions)
        
        action = action.view(T, B)

        return (
            dict(policy_logits=policy_logits,  action=action),
            core_state,
        )
"""agent"""

# Main

In [ ]:
def main(model_name = None):
    # Seed
    seeds = [1,2,3,4,5]

    # Initialise environment
    env = gym.make("NetHackScore-v0",observation_keys=("glyphs", "blstats"))
    env.seed(10)
    hyper_params = {
        "replay-buffer-size": 1000,  # replay buffer size
        "learning-rate": 0.00048,  # learning rate for RMSprob
        "discount-factor": 0.99,  # discount factor
        "num-steps": int(1e7),  # total number of steps to run the environment for
        "batch-size": 256,  # number of transitions to optimize at the same time
        "learning-starts": 10000,  # number of steps before learning starts
        "learning-freq": 5,  # number of iterations between every optimization step
        "use-double-dqn": True,  # use double deep Q-learning
        "target-update-freq": 1000,  # number of iterations between every target network update
        "eps-start": 1.0,  # e-greedy start threshold
        "eps-end": 0.01,  # e-greedy end threshold
        "eps-fraction": .5,  # fraction of num-steps
        "print-freq":1,
        "save-freq":2,
        "save-name":'dqn_2',
        "model_name": model_name
    }
    rewards,mean_rewards,loss,path = train_dqn(env,hyper_params)

    plt.figure(figsize=(15,15))
    plt.plot(rewards)
    plt.ylabel("reward")
    plt.xlabel("episode")
    plt.savefig('img/nethack_dqn_reward.png')
    
    plt.figure(figsize=(15,15))
    plt.plot(mean_rewards)
    plt.ylabel("mean 100 episode reward")
    plt.xlabel("episode")
    plt.savefig('img/nethack_dqn_mean_reward.png')
    
    plt.figure(figsize=(15,15))
    plt.plot(loss)
    plt.ylabel("loss")
    plt.savefig('img/nethack_dqn_loss.png')
    
    #Number of times each seed will be run
    num_runs = 10

    #Run a few episodes on each seed
    rewards = []
    for seed in seeds:
        env.seed(seed)
        seed_rewards = []
        for i in range(num_runs):
            seed_rewards.append(run_episode(env))
        rewards.append(np.mean(seed_rewards))

    # Close environment and print average reward
    env.close()
    print("Average Reward: %f" %(np.mean(rewards)))


# Train DQN

In [ ]:
def train_dqn(env,hyper_params):
    


    replay_buffer = ReplayBuffer(hyper_params["replay-buffer-size"])
    agent=DQNAgent(env.observation_space,env.action_space.n,replay_buffer,\
                   hyper_params["learning-rate"],hyper_params["batch-size"],hyper_params["discount-factor"])
    
    eps_timesteps = hyper_params["eps-fraction"] * float(hyper_params["num-steps"])
    episode_rewards = [torch.zeros(1, 1)]
    num_episode_step_counter = [0]

    episode_loss_len = 0
    episode_loss = defaultdict(lambda: torch.zeros(1, 1))
    episode_loss_av = {}
    train_loss = {}
    t=0
    
    if hyper_params['model_name'] is not None:
        checkpoints = load_checkpoint(hyper_params['model_name'], agent)
        step_num = checkpoints['step_num']
        episode_rewards = checkpoints['episode_rewards'] + [torch.zeros(1, 1)]
        train_loss = checkpoints['train_loss']
        t = checkpoints['step_num']+1
        
        
        ##commentend out as old checkpont function foirgot to save these but took them as imputs
        ##uncomment if loading a new dqn 
        # episode_loss = checkpoints['episode_loss']
        # episode_loss_av = checkpoints['episode_loss_av']
        # num_episode_step_counter=checkpoints['num_episode_step_counter'] + [0] #  not even sure why we have this. 
        # if episode_loss ==None:
        #     episode_loss = defaultdict(lambda: torch.zeros(1, 1))
        # if episode_loss_av==None:
        #     pisode_loss_av = {}
       

    state = env.reset()
    state=format_observations(state)
    done = torch.ones(1, 1, dtype=torch.uint8)
    pbar = tqdm((range(t, hyper_params["num-steps"])))
    for t in (range(t,hyper_params["num-steps"])):
        fraction = min(1.0, float(t) / eps_timesteps)
        eps_threshold=hyper_params["eps-start"]+fraction*(hyper_params["eps-end"]-hyper_params["eps-start"])
        sample = random.random()
        if sample <= eps_threshold:
            action = env.action_space.sample()
        else:
            action = agent.act(state, done)

        #take step in env
        state_prime,reward,done,_ = env.step(action)
        # clip rewards
        # reward = np.tanh(reward/100)
        # reward = torch.clamp(reward, -1, 1)
        action=torch.tensor(action)
        reward=torch.tensor(reward).view(1, 1)
        done=torch.tensor(done).view(1, 1)
        next_state=format_observations(state_prime)
        agent.replay_buffer.add(state,action,reward,next_state, (~done))
        state = next_state.copy()
        episode_rewards[-1] += reward
        num_episode_step_counter[-1] += 1
        num_episodes = len(episode_rewards)

        if (t>hyper_params["learning-starts"] and t%hyper_params["learning-freq"]==0):
            train_loss[t] = (agent.optimise_td_loss())
            episode_loss[len(episode_rewards)] += train_loss[t]
            episode_loss_len += 1
        
        if (t>hyper_params["learning-starts"] and len(episode_rewards)%hyper_params["print-freq"]==0 and done):
            mean_100ep_reward = round(np.mean(episode_rewards[-25:-1]), 1)
            mean_100ep_loss = round(np.mean(list(train_loss.values())[-25:-1]), 1)
            # pbar.set_description(
            #     "steps: {}".format(t) +
            #     " episodes: {}".format(num_episodes) +
            #     " ep reward: {}".format(episode_rewards[-1]) +
            #     " last loss: {}".format((train_loss[-1])) +
            #     (f" mean_reward: {mean_100ep_reward:.4f}") +
            #     (f" mean_loss: {mean_100ep_loss:.7f}") +
            #     " explore: {}".format(int(100 * eps_threshold))
            # )
            print(
                "steps: {}".format(t),
                "episodes: {}".format(num_episodes),
                "ep reward: {}".format(episode_rewards[-1]),
                "last loss: {}".format((list(train_loss.values())[-1])),
                (f"mean_reward: {mean_100ep_reward:.4f}"),
                (f"mean_loss: {mean_100ep_loss:.7f}"),
                "explore: {}".format(int(100 * eps_threshold)))
            
        if (len(episode_rewards) % hyper_params['save-freq'] == 0 and done):
            save_checkpoint(
                path(hyper_params['save-name']+f'-{len(episode_rewards)}.pt'),
                agent, t, episode_rewards, num_episode_step_counter,
                train_loss, episode_loss, episode_loss_av)

        if done:
            state = env.reset()
            state = format_observations(state)
            if (t > hyper_params["learning-starts"]):
                episode_loss_av[len(episode_rewards)] = (episode_loss[len(episode_rewards)]/episode_loss_len)
            episode_rewards.append(torch.zeros(1, 1))
            num_episode_step_counter.append(0)
            episode_loss_len = 0
            # print("length of ep reward", len(episode_rewards))

        if (t>hyper_params["learning-starts"] and t%hyper_params["target-update-freq"]==0):
            agent.update_target_network()

    PATH = 'models/dqn_nethack.pth'
    torch.save(agent.dqn.state_dict(), PATH)
    return episode_rewards.copy(),mean_100ep_reward.copy(),train_loss.copy(),PATH


# Run

In [ ]:
main('dqn_2-528.pt')

In [ ]:
def plot_episode(episode, fname, label):
    np.save(path(fname), episode)
    plt.figure(figsize=(12,8))
    plt.plot(np.arange(len(episode)), episode)
    plt.xlabel('Training episode')
    plt.ylabel(F'The {label} per episode')
    plt.title(F' Nethack Training curves tracking the agent’s {label} per episode')
    plt.show()
    plt.savefig(path(fname + '.png'))

def plot_loss(losses, fname, label, label2):
    # redundent function (to an extent)
    np.save(path(fname), list(losses.values()))
    plt.figure(figsize=(12,8))
    plt.plot(list(losses.keys()), list(losses.values()))
    plt.xlabel(F'Training {label2}')
    plt.ylabel(F'The {label} per {label2}')
    plt.title(F'Training curves tracking the {label} per {label2}')
    plt.show()
    plt.savefig(path('steps-loss.png'))

In [ ]:
checkpoints = load_checkpoint('dqn_1-125.pt', agent)

In [ ]:
checkpoints = load_checkpoint('dqn_1-125.pt', agent)
step_num = checkpoints['step_num']
episode_rewards = checkpoints['episode_rewards']
train_loss = checkpoints['train_loss']
train_loss = checkpoints['train_loss']

In [ ]:
plot_episode(np.array(episode_rewards[:-1]), 'episode-rewards', 'score')
# plot_loss(episode_loss_av, 'episode-loss', 'average loss', 'episode')
# plot_loss(episode_loss, 'average-episode-loss', 'loss', 'episode')
plot_loss(train_loss, 'steps-loss', 'loss', 'step')

# To Do:


* Write the report
* Actor Critic





